In [27]:
import os
import csv
from collections import defaultdict
import json

# To finetune

In [ ]:
DATASET_VAR = 'augm_final'        # augm_final_1, base_final_1, orig
DATASET_NAME = 'dataset'
RESULTS_PATH = f'/home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/{DATASET_VAR}'
VALID_TABLE = f'{RESULTS_PATH}/valid/BirdNET_SelectionTable.txt'
TRAIN_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/train'
VALID_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/valid'
TEST_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/test'
TEST_TABLE = f'{RESULTS_PATH}/test/BirdNET_SelectionTable.txt'
FINAL_RESULTS_PATH = '/home/giacomoschiavo/finetuning-BirdNET/models/results'

In [159]:
print(f'python -m birdnet_analyzer.train --i {TRAIN_PATH} --o {RESULTS_PATH}/{DATASET_VAR}.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file {RESULTS_PATH}/{DATASET_VAR}.npz')

python -m birdnet_analyzer.train --i /home/giacomoschiavo/segments/dataset/train --o /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/augm_final/augm_final.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/augm_final/augm_final.npz


# To analize

In [160]:
MIN_CONF = '0.05'

In [ ]:
print(f'python -m birdnet_analyzer.analyze --i {VALID_PATH} --o {RESULTS_PATH}/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite --sensitivity 1')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/valid --o /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/augm_final/valid_0_5 --slist /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/augm_final/augm_final.tflite --sensitivity 0.5


In [ ]:
print(f'python -m birdnet_analyzer.analyze --i {TEST_PATH} --o {RESULTS_PATH}/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite --sensitivity 1')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/test --o /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/augm_final/test_0_5 --slist /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/augm_final/augm_final.tflite --sensitivity 0.5


# Analysis

In [163]:
with open(f"./utils/{DATASET_NAME}/dataset_config_custom.json") as f:
    dataset_config = json.load(f)
class_names = list(dataset_config['mappings'].keys())

In [164]:
with open('/home/giacomoschiavo/finetuning-BirdNET/utils/species_dict_map.json') as f:
    species_dict = json.load(f)
inv_species_dict = {value: key for key, value in species_dict.items()}

In [165]:
def get_conf_scores():
    conf_scores = {}

    #  {'Aeroplane': [(np.float32(0.0), False),
    #               (np.float32(1.3937646e-30), False),
    #               (np.float32(1.0654355e-25), False),
    #               (np.float32(0.0), False),

    with open(VALID_TABLE, 'r') as f:
        reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
        for row in reader:
            file_path = row['Begin Path']
            pred_species_name = row['Common Name']
            if row['Common Name'] in inv_species_dict:
                pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
            true_species_name = file_path.split('/')[-2]
            confidence = float(row['Confidence'])
            if pred_species_name not in conf_scores:
                conf_scores[pred_species_name] = []
            is_correct = pred_species_name == true_species_name
            conf_scores[pred_species_name].append((confidence, is_correct))
    return conf_scores
            

In [166]:
conf_scores = get_conf_scores()

In [167]:
from sklearn.metrics import f1_score
import numpy as np 

DEFAULT_THRESH = 0.15

def compute_best_thresholds(conf_scores, num_thresholds=200, min_thresh=0.01, max_thresh=0.95):
    thresholds = {}

    for species, values in conf_scores.items():
        probs, truths = zip(*values)
        probs = np.array(probs)
        truths = np.array(truths).astype(int)

        best_thresh = DEFAULT_THRESH
        best_f1 = 0.0

        for thresh in np.linspace(min_thresh, max_thresh, num_thresholds):
            preds = (probs >= thresh).astype(int)
            f1 = f1_score(truths, preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        thresholds[species] = best_thresh
        print(f"📊 {species} -> {best_thresh:.3f}, F1-score: {best_f1:.3f}")

    return thresholds



In [168]:
best_thresholds = compute_best_thresholds(conf_scores)

📊 Aeroplane -> 0.010, F1-score: 0.625
📊 Fringilla coelebs_Common Chaffinch -> 0.308, F1-score: 0.635
📊 Phylloscopus collybita_Common Chiffchaff -> 0.109, F1-score: 0.564
📊 Sylvia atricapilla_Eurasian Blackcap -> 0.142, F1-score: 0.610
📊 Turdus merula_Eurasian Blackbird -> 0.246, F1-score: 0.616
📊 Anthus trivialis_Tree Pipit -> 0.520, F1-score: 0.648
📊 Erithacus rubecula_European Robin -> 0.180, F1-score: 0.684
📊 Parus major_Great Tit -> 0.600, F1-score: 0.615
📊 None -> 0.341, F1-score: 0.895
📊 Periparus ater_Coal Tit -> 0.251, F1-score: 0.721
📊 Certhia familiaris_Eurasian Treecreeper -> 0.841, F1-score: 0.745
📊 Regulus ignicapilla_Common Firecrest -> 0.133, F1-score: 0.525
📊 Regulus regulus_Goldcrest -> 0.010, F1-score: 0.601
📊 Troglodytes troglodytes_Eurasian Wren -> 0.185, F1-score: 0.565
📊 Wind -> 0.128, F1-score: 0.578
📊 Lophophanes cristatus_Crested Tit -> 0.369, F1-score: 0.780
📊 Vegetation -> 0.459, F1-score: 0.771
📊 Coccothraustes coccothraustes_Hawfinch -> 0.010, F1-score: 0.8

In [169]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_test.json') as f:
    true_segments_test = json.load(f)

In [170]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_train.json') as f:
    true_segments_full = json.load(f)

In [171]:
true_segments_full.update(true_segments_test)

In [172]:
from sklearn.preprocessing import MultiLabelBinarizer

test_species_list = os.listdir(TEST_PATH)
# test_species_list = [species for species in test_species_list if len(species.split('_')) > 1]
mlb = MultiLabelBinarizer()
mlb.fit([test_species_list])
# if DATASET_VAR == "orig":
#     class_names = [species for species in test_species_list if len(species.split("_")) > 1]
#     mlb.fit([class_names])

len(mlb.classes_)

20

In [173]:
pred_segments_proba = {}
with open(TEST_TABLE, 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
    for row in reader:
        file_path = row['Begin Path']
        audio_name = os.path.basename(file_path)    # in test non sono presenti WABAD o augm
        only_audio_name = "_".join(audio_name.split("_")[:2]) + ".WAV"
        segm = "_".join(audio_name.split(".")[0].split("_")[-2:])
        pred_species_name = row['Common Name']
        if row['Common Name'] in inv_species_dict:
            pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
        # if len(file_path.split('/')[-2].split('_')) == 1:       # skip if None, Pecking, etc. 
        #     continue
        confidence = float(row['Confidence'])
        pred_segments_proba.setdefault(only_audio_name, {})
        best_thresholds.setdefault(pred_species_name, DEFAULT_THRESH)
        if confidence >= best_thresholds[pred_species_name]:
            pred_segments_proba[only_audio_name].setdefault(segm, {})
            if "None" in pred_segments_proba[only_audio_name][segm]:
                continue
            if pred_species_name == "None":
                pred_segments_proba[only_audio_name][segm] = {"None": confidence}
            else:
                pred_segments_proba[only_audio_name][segm].update({pred_species_name: confidence})

In [174]:
true_segments = defaultdict(dict)
for species in os.listdir(TEST_PATH):
    if species not in test_species_list:
        print(species, 'ignored')
        continue
    for audio in os.listdir(os.path.join(TEST_PATH, species)):
        audio = audio.split('.')[0]
        date, time, segm1, segm2 = audio.split('_')
        audio_name = '_'.join([date, time]) + '.WAV'
        segm = '_'.join([segm1, segm2])
        if segm not in true_segments[audio_name]:
            true_segments[audio_name][segm] = []
        true_segments[audio_name][segm].extend([species])

In [175]:
for audio in true_segments.keys():
    pred_segments_proba.setdefault(audio, {})
    if audio in pred_segments_proba:
        for segm in true_segments[audio].keys():
            pred_segments_proba[audio].setdefault(segm, {})

In [176]:
# extract recognized labels
pred_segments = {}
pred_proba = {}

for audio, segments in pred_segments_proba.items():
    pred_segments.setdefault(audio, {})
    pred_proba.setdefault(audio, {})
    for segm, labels in segments.items():
        pred_segments[audio].setdefault(segm, {})
        pred_segments[audio][segm] = list(labels.keys())
        pred_proba[audio].setdefault(segm, {})
        pred_proba[audio][segm] = list(labels.values())

In [177]:
def segment_key_to_float(k):
    return float(k.replace("_", "."))

def get_conf_dict(pred_segments):
    for audio in pred_segments.keys():
        pred_segments[audio] = dict(sorted(pred_segments[audio].items(), key=lambda x: segment_key_to_float(x[0]))) 
    pred_segments = dict(sorted(pred_segments.items(), key=lambda x: x[0])) 
    return pred_segments

In [178]:
pred_segments_proba = get_conf_dict(pred_segments_proba)

In [ ]:
with open(f"{RESULTS_PATH}/test_pred_segments.json", 'w') as f:
    json.dump(pred_segments_proba, f, indent=4)

In [180]:
y_pred = []
y_true = []
y_pred_proba = []

for audio in pred_segments:
    for segment in sorted(pred_segments[audio].keys()):
        true_labels = true_segments[audio].get(segment, [])
        pred_labels = pred_segments[audio].get(segment, [])
        proba_values = pred_proba[audio].get(segment, [])

        y_true_vec = mlb.transform([true_labels])[0]  # 1D array
        y_pred_vec = mlb.transform([pred_labels])[0]  # 1D array

        proba_vec = np.zeros(len(mlb.classes_))
        for label, score in zip(pred_labels, proba_values):
            if label in mlb.classes_:
                idx = list(mlb.classes_).index(label)
                proba_vec[idx] = score

        y_true.append(y_true_vec)
        y_pred.append(y_pred_vec)
        y_pred_proba.append(proba_vec)

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_pred_proba = np.array(y_pred_proba)


In [181]:
from sklearn.metrics import classification_report
import pandas as pd
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)

report_df = pd.DataFrame(report).T
report_df

,precision,recall,f1-score,support
Aeroplane,0.000000,0.000000,0.000000,22.0
Anthus trivialis_Tree Pipit,0.586420,0.558824,0.572289,170.0
Certhia familiaris_Eurasian Treecreeper,0.672131,0.350427,0.460674,117.0
Coccothraustes coccothraustes_Hawfinch,0.705882,0.137931,0.230769,87.0
Erithacus rubecula_European Robin,0.561508,0.508993,0.533962,556.0
Fringilla coelebs_Common Chaffinch,0.585778,0.654808,0.618373,1799.0
Lophophanes cristatus_Crested Tit,0.250000,0.230769,0.240000,26.0
Loxia curvirostra_Common Crossbill,0.215686,0.478261,0.297297,46.0
Muscicapa striata_Spotted Flycatcher,0.000000,0.000000,0.000000,173.0
None,0.907738,0.651292,0.758423,4683.0


In [ ]:
with open(f'{RESULTS_PATH}/classification_report.json', 'w') as f:
    json.dump(report, f)

In [ ]:
np.savez(f'{FINAL_RESULTS_PATH}/results_b{DATASET_VAR}.npz', y_true=y_true, y_pred=y_pred, y_pred_proba=y_pred_proba, class_names=mlb.classes_)